### Include the important libraries

In [1]:
import csv
import math
import random
import seaborn
import benchmark
import numpy as np
import pandas as pd

### Iterated Local Search (ILS)

#### Generate a initial solution

In [2]:
def initial_solution(lower_bound, upper_bound):
    return np.random.uniform(lower_bound, upper_bound, size=2)    

#### Calculate cost function

In [3]:
def calculate_cost(solution, num_func):
    # Fazer uma função de calculo de custo puxando a get_func_details dela
    x = solution[0]
    y = solution[1]
    if num_func == 0 or num_func == 1:
        cost = benchmark.func1_op(x, y)
    elif num_func == 2 or num_func == 3:
        cost = benchmark.func2_op(x, y)
        
    return cost

#### Local search function was made based on swap

In [4]:
def local_search(solution, num_func):
    best_solution = solution.copy()  # Crie uma cópia da solução atual como a melhor solução
    best_cost = calculate_cost(solution, num_func)  # Calcule o custo da melhor solução
    
    improved = True
    while improved:
        improved = False

        for i in range(len(solution)):
            for j in range(i + 1, len(solution)):
                # Troque os elementos i e j na solução
                new_solution = solution.copy()
                new_solution[i], new_solution[j] = new_solution[j], new_solution[i]

                # Calcule o custo da nova solução
                new_cost = calculate_cost(new_solution, num_func)

                # Se a nova solução for melhor, atualize a melhor solução
                if new_cost < best_cost:
                    best_solution = new_solution
                    best_cost = new_cost
                    improved = True

        # Atualize a solução atual para a melhor solução encontrada nesta iteração
        solution = best_solution

    return [best_cost, best_solution]

#### Perturbation function

In [5]:
def perturbation(solution, lower_bound, upper_bound):
    perturbed_solution = solution.copy()
    
    # Aplica a perturbação em cada variável
    for i in range(len(solution)):
        perturbed_solution[i] += random.uniform(lower_bound, upper_bound)
    
    return perturbed_solution

#### Iterated Local Search algorithm

In [6]:
def ILS(num_iter, lb, ub, num_func):
    initial_best = initial_solution(lb, ub)
    best_cost, best_solution = local_search(initial_best, num_func)
    i = 0
    while (i < num_iter):
        perturbed = perturbation(best_solution, lb, ub)
        local_cost, local_solution = local_search(perturbed, num_func)
        if local_cost < best_cost:
            best_cost, best_solution = local_cost, local_solution
        i+=1
    return best_cost

### Hill Climbing

In [12]:
def hillclimbing(objf, bounds, num_iter, step_size):
    solution = bounds[:, 0] + np.random.rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
    solution_eval = objf(solution[0], solution[1])

    for i in range(num_iter):
        candidate = solution + np.random.rand(len(bounds)) * step_size
        candidate_eval = objf(candidate[0], candidate[1])

        if candidate_eval <= solution_eval:
            solution, solution_eval = candidate, candidate_eval
        #print(">%d f(%s) = %.5f" %(i, solution, solution_eval))
    
    return solution_eval

### Creation of a CSV file to get the function results

In [30]:
data_bench = "./data/f2_2.csv"
f_min_ils = math.inf
f_min_hc = math.inf
bench_value_ils = []
bench_value_hc = []
fields = ['Algoritmo', 'Mínimo', 'Máximo', 'Média', 'Desvio-padrão']

### Main code

In [31]:
# Number of iterations
num_iter = 100
# function number:
# 0 = func1_op -5 < x,y < 5
# 1 = func1_op -2 < x,y < 2
# 2 = func2_op -512 < x,y < 512
# 3 = func2_op 400 < x,y < 512
num_func = 3

fobj, lower_bound, upper_bound = benchmark.get_func(3)

bounds = np.asarray([[lower_bound, upper_bound], [lower_bound, upper_bound]])

step_size = 0.05

with open(data_bench, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    for i in range(30):
        # ILS algorithm
        bench_value_ils.append(ILS(num_iter, lower_bound, upper_bound, num_func))
        # Hillclimbing algorithm
        bench_value_hc.append(hillclimbing(getattr(benchmark, 'func2_op'), bounds, num_iter, step_size))
    bench_dict_ils = [{'Algoritmo': 'ILS', 'Mínimo':np.min(bench_value_ils), 'Máximo':np.max(bench_value_ils),'Média':np.mean(bench_value_ils), 'Desvio-padrão':np.std(bench_value_ils)}]
    bench_dict_hc = [{'Algoritmo': 'Hill Climbing', 'Mínimo':np.min(bench_value_hc), 'Máximo':np.max(bench_value_hc),'Média':np.mean(bench_value_hc), 'Desvio-padrão':np.std(bench_value_hc)}]
    writer.writerows(bench_dict_ils)
    writer.writerows(bench_dict_hc)
    csvfile.close()